In [ ]:
from shapely.geometry import box
import osmnx as ox

ox.settings.cache_folder = "../data/cache"

In [9]:
size = 0.3
centroid = (4.33356080866911, 50.85864602196642)
x1y1 = (coord - size /  2 for coord in centroid)
x2y2 = (coord + size / 2 for coord in centroid)
bbox = box(*x1y1, *x2y2)

In [10]:
gdf = ox.features.features_from_polygon(bbox, tags={"landuse": True})
gdf = gdf.reset_index()